In [22]:
from dotenv import load_dotenv
load_dotenv()
import os
import torch
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec
from transformers import AutoTokenizer, AutoModel


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Users/sufyan/opt/anaconda3/envs/rag/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/sufyan/opt/anaconda3/envs/rag/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/sufyan/opt/anaconda3/envs/rag/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/sufyan/opt/anaconda3/envs/rag/lib/python3.10/site-packages/traitlets/config/application.py",

In [34]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", 
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '5b3c6baafca7ccd287c2c4fc68370b9c', 'Date': 'Sun, 25 Aug 2024 22:02:48 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [18]:
import json
data = json.load(open("reviews.json"))
data["reviews"]

[{'professor': 'Dr. Jane Doe',
  'subject': 'Physics',
  'stars': 5,
  'review': 'Amazing professor! Makes complex topics easy to understand.'},
 {'professor': 'Dr. John Smith',
  'subject': 'Mathematics',
  'stars': 4,
  'review': 'Very knowledgeable but sometimes hard to follow.'},
 {'professor': 'Dr. Alice Johnson',
  'subject': 'Chemistry',
  'stars': 3,
  'review': 'Good content, but lectures are a bit dry.'},
 {'professor': 'Dr. Robert Brown',
  'subject': 'Computer Science',
  'stars': 5,
  'review': 'Engaging and practical. One of the best CS professors!'},
 {'professor': 'Dr. Emily Davis',
  'subject': 'Biology',
  'stars': 2,
  'review': 'Too much reading and not enough explanation.'},
 {'professor': 'Dr. Michael Wilson',
  'subject': 'History',
  'stars': 4,
  'review': 'Interesting lectures, but exams are tough.'},
 {'professor': 'Dr. Sarah Miller',
  'subject': 'Economics',
  'stars': 3,
  'review': 'Average professor, decent lectures.'},
 {'professor': 'Dr. Daniel Anderso

In [29]:
processed_data = []
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

for review in data["reviews"]:
    inputs = tokenizer(review['review'], return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    embedding = torch.mean(outputs.last_hidden_state, dim=1).squeeze().tolist()
    
    processed_data.append(
        {
            "values": embedding,
            "id": review["professor"],
            "metadata":{
                "review": review["review"],
                "subject": review["subject"],
                "stars": review["stars"],
            }
        }
    )

/Users/sufyan/opt/anaconda3/envs/rag/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [30]:
processed_data[0]

{'values': [-0.1510464996099472,
  0.1422683596611023,
  -0.06996234506368637,
  -0.06696899980306625,
  -0.24244903028011322,
  -0.29410770535469055,
  0.035952161997556686,
  0.01380439754575491,
  -0.0538240522146225,
  0.11732327193021774,
  -0.1681586652994156,
  0.33571362495422363,
  -0.23730643093585968,
  0.4143395721912384,
  -0.1001080796122551,
  -0.07381176948547363,
  -0.1153927743434906,
  -0.1937093734741211,
  -0.346631795167923,
  -0.42381536960601807,
  0.1563255935907364,
  0.2275293618440628,
  0.11645998805761337,
  -0.31045788526535034,
  -0.11558296531438828,
  0.12664909660816193,
  -0.029945457354187965,
  -0.09121561050415039,
  0.5156161189079285,
  -0.3438089191913605,
  -0.14175739884376526,
  0.47835132479667664,
  0.06757547706365585,
  0.14122770726680756,
  -0.17098529636859894,
  0.09991606324911118,
  0.030365079641342163,
  0.5542579293251038,
  0.26296520233154297,
  0.02020135521888733,
  -0.3231176435947418,
  0.17470693588256836,
  0.23491926491

In [35]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 20}

In [36]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}